In [2]:
import numpy as np
import pandas as pd # for loading data

In [3]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [4]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [5]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [6]:
train_df = pd.read_csv("train.csv")

## Cleaning the test set lyrics and creating One-Hots for train data's genres and just cleaned lyrics

In [7]:
lyrics, word_list = clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [8]:
train_genres = one_hot_genres(train_df['Genre'])

## Replace all the lyrics with their respctive vector of word occurrences to be used as the input layer for the neural network

In [9]:
train_lyrics = np.zeros(shape=(len(train_df),len(word_list)))
for index, value in train_df.iterrows():
    train_lyrics[index] = one_hot(str(train_df['Lyrics'][index]), word_list, len(word_list), 0)
print(train_lyrics.shape)
lyrics_size = len(word_list)

# Cleaning no more used variables
del train_df

(12240, 37818)


## Cleaning the test set lyrics and creating One-Hots for test data's genres and just cleaned lyrics

In [10]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_list = clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [11]:
test_genres = one_hot_genres(test_df['Genre'])

In [12]:
test_lyrics = np.zeros(shape=(len(test_df), lyrics_size))
for index, value in test_df.iterrows():
    test_lyrics[index] = one_hot(str(test_df['Lyrics'][index]), word_list, lyrics_size, 0)
print(test_lyrics.shape)

# Cleaning no more used variables
del test_df

(2160, 37818)


# Neural Network stuff 
## **Modify and run only the cells below to change the neural network**

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(units=256, input_shape=(lyrics_size,)))
model.add(Activation('relu'))            # activation layer
model.add(Dropout(0.8))
model.add(Dense(32))
model.add(Dense(8))
model.add(Activation('softmax'))         # output class probabilities

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1080)              40844520  
_________________________________________________________________
activation (Activation)      (None, 1080)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1080)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               276736    
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 2056      
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
Tota

In [14]:
from tensorflow.keras.optimizers import SGD, Adam

optimizer = Adam(lr=0.0001) # lr is the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(train_lyrics, train_genres, epochs=8, validation_split=17/100, batch_size=16);

Train on 10159 samples, validate on 2081 samples
Epoch 1/10
10159/10159 [==============================] - 68s 7ms/sample - loss: 1.7108 - acc: 0.3690 - val_loss: 1.4538 - val_acc: 0.4853
Epoch 2/10
10159/10159 [==============================] - 68s 7ms/sample - loss: 1.2398 - acc: 0.5624 - val_loss: 1.3195 - val_acc: 0.5252
Epoch 3/10
10159/10159 [==============================] - 68s 7ms/sample - loss: 0.9429 - acc: 0.6857 - val_loss: 1.3013 - val_acc: 0.5238
Epoch 4/10
10159/10159 [==============================] - 67s 7ms/sample - loss: 0.7432 - acc: 0.7566 - val_loss: 1.3152 - val_acc: 0.5257
Epoch 5/10
10159/10159 [==============================] - 67s 7ms/sample - loss: 0.5819 - acc: 0.8201 - val_loss: 1.3640 - val_acc: 0.5286
Epoch 6/10
 1952/10159 [====>.........................] - ETA: 52s - loss: 0.4268 - acc: 0.8786

KeyboardInterrupt: 

## Testing the Neural Network's performace + Confusion Matrix

In [16]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)

2160/2160 [==============================] - 2s 779us/sample - loss: 1.4103 - acc: 0.5190


In [ ]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(test_lyrics)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values